In [8]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# from MCMAC_pre_merger import MCengine_pre_merger
# from MCMAC_post_merger import MCengine_post_merger 

# change prior: angle 0-45 degree
from modules.MCMAC_pre_merger_changeprior_45degree import MCengine_pre_merger
from modules.MCMAC_post_merger_changeprior_45degree import MCengine_post_merger 

import numpy as np
import pickle

# Input A2034 parameters

In [9]:
pre = False
post = True

change_prior_angle = True

# Setting simulation parameters (define prior distributions)
N_mc = 1000  # Number of Monte Carlo samples

cluster_name ='A2034'

if change_prior_angle is True:
    folder_name = cluster_name + '_0-45degree'
else:
    folder_name = cluster_name

arcsecond = 300
redshift = 0.114

# (mean value, standard deviation)
M1 = (2.35e14, 0.84e14)  # Mass distribution of A2034NW (Subcluster 1)
M2 = (1.08e14, 0.51e14)  # Mass distribution of A2034NE (Subcluster 2)
Z1 = (0.114, 0.0001)  # Redshift distribution of A2034NW (Subcluster 1)
Z2 = (0.114, 0.0001)  # Redshift distribution of A2034NE (Subcluster 2)

TSC_simulation = 0.26

# when you use pre-MCMAC code
if pre is True:
    prefix = "../RealClusters/" + folder_name + "/simulation_pre_output" 

# when you use post-MCMAC code
if post is True:
    prefix = "../RealClusters/" + folder_name + "/simulation_post_output" 

result_file = "result_" + folder_name + ".csv"

## arcsecond to kpc

In [10]:
from astropy.cosmology import Planck18 as cosmo

def arcsecondToKpc(redshift, arcsecond):
    # Convert arcseconds to kiloparsecs
    kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60  # kpc per arcsec
    distance_kpc = arcsecond * kpc_per_arcsec
    return distance_kpc

In [11]:

distance_Mpc = arcsecondToKpc(redshift, arcsecond) * 0.001 # kpc to Mpc
print(distance_Mpc)

D_proj = (distance_Mpc, 0.05*distance_Mpc, 0.05*distance_Mpc)  # Projected separation distance distribution (mean 408 kpc, standard deviation 14 kpc)

0.6415971364028084


## Confirm it collision or not

In [12]:
# Execute simulation (derive posterior distributions of dynamical properties such as collision velocity, time since collision, etc.)
# MCengine_pre_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Probability of system being bound = 1.00

Probability of system being boud is over 50%, then use post merger code.

In [13]:
MCengine_post_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Completed Monte Carlo iteration 10 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 100 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 1000 of 1000.
~0 minutes remaining


(array([1.23948177e+14, 2.36490213e+14, 4.22423244e+14, 2.85113771e+14,
        2.23943758e+14, 4.93362011e+14, 2.63344482e+14, 3.76258987e+14,
        1.86676218e+14, 2.84144388e+14, 8.50379280e+13, 1.45416479e+14,
        2.18792726e+14, 1.58382496e+14, 2.62459465e+14, 2.39638530e+14,
        3.92699349e+14, 2.21032024e+14, 2.14701704e+14, 1.96071305e+14,
        1.63664663e+14, 3.07647518e+14, 2.31932611e+14, 2.43035613e+14,
        1.88454707e+14, 1.15303677e+14, 4.30329333e+14, 1.49232205e+14,
        2.16327891e+14, 2.05313943e+14, 1.39369454e+14, 3.94746134e+14,
        2.48126666e+14, 1.76387648e+14, 2.69769903e+14, 3.18755503e+14,
        2.25440267e+14, 1.11537785e+14, 1.86733217e+14, 1.85777565e+14,
        1.22389111e+14, 2.48993329e+14, 1.75426256e+14, 3.10831533e+14,
        3.08189917e+14, 3.16993963e+14, 2.33289450e+14, 1.73219261e+14,
        2.53685950e+14, 3.27096572e+14, 2.12340638e+14, 3.25809602e+14,
        2.60621824e+14, 1.63359851e+14, 3.67458920e+14, 2.110281

In [14]:
import pickle
import numpy as np
import pandas as pd

# 単位の定義
units = {
    'm1': '10^14 M☉',
    'm2': '10^14 M☉',
    'z1': '',
    'z2': '',
    'dproj': 'Mpc',
    'vradobs': 'km s^{-1}',
    'alpha': 'degree',
    'v3dobs': 'km s^{-1}',
    'd3d': 'Mpc',
    'v3dcol': 'km s^{-1}',
    'dmax': 'Mpc',
    'TSM0': 'Gyr',
    'TSM1': 'Gyr',
    'T': 'Gyr',
    'prob': ''
}

# パラメータファイルのリスト
parameters = ['_m_1.pickle', '_m_2.pickle', '_z_1.pickle', '_z_2.pickle', '_d_proj.pickle', 
              '_v_rad_obs.pickle', '_alpha.pickle', '_v_3d_obs.pickle', '_d_3d.pickle', 
              '_v_3d_col.pickle', '_d_max.pickle', '_TSM_0.pickle', '_TSM_1.pickle', 
              '_T.pickle', '_prob.pickle']

results = []

for param in parameters:
    with open(prefix + param, 'rb') as f:
        data = pickle.load(f)
        if param in ['_m_1.pickle', '_m_2.pickle']:
            # 質量をe14単位に変換
            data = np.array(data) / 1e14
        
        # Medianを計算
        median = np.median(data)
        
        # 68%および95%の信頼区間を計算
        lcl_68, ucl_68 = np.percentile(data, [16, 84])
        lcl_95, ucl_95 = np.percentile(data, [2.5, 97.5])
        
        param_name = param.replace('_', '').replace('.pickle', '')
        
        # 数値を有効数字2桁にフォーマット
        if param_name not in ['z_1', 'z_2']:
            median = round(median, 2)
            lcl_68 = round(lcl_68, 2)
            ucl_68 = round(ucl_68, 2)
            lcl_95 = round(lcl_95, 2)
            ucl_95 = round(ucl_95, 2)
        
        results.append({
            'Parameter': param_name,
            'Units': units[param_name],
            'Median': median,
            '68% LCL': lcl_68,
            '68% UCL': ucl_68,
            # '95% LCL': lcl_95,
            # '95% UCL': ucl_95
        })

# 結果をDataFrameに変換
df_results = pd.DataFrame(results)
display(df_results)

# TSM0とTSM1に対応する行の抽出
tsm0_row = df_results[df_results['Parameter'] == 'TSM0']
tsm1_row = df_results[df_results['Parameter'] == 'TSM1']

# CSVに出力するための新しい行を作成
new_row = {
    'ClusterName': cluster_name,
    'MCMAC_TSC0': tsm0_row['Median'].values[0],
    'MCMAC_TSC0.lower': tsm0_row['68% LCL'].values[0],
    'MCMAC_TSC0.upper': tsm0_row['68% UCL'].values[0],
    'MCMAC_TSC1': tsm1_row['Median'].values[0],
    'MCMAC_TSC1.lower': tsm1_row['68% LCL'].values[0],
    'MCMAC_TSC1.upper': tsm1_row['68% UCL'].values[0],
    'Simulation_TSC': TSC_simulation, 
}

# 新しい行をDataFrameに変換
new_row_df = pd.DataFrame([new_row])

# CSVファイルに出力
new_row_df.to_csv(result_file, index=False)


,Parameter,Units,Median,68% LCL,68% UCL
0,m1,10^14 M☉,2.40,1.53,3.21
1,m2,10^14 M☉,1.12,0.60,1.61
2,z1,,0.11,0.11,0.11
3,z2,,0.11,0.11,0.11
4,dproj,Mpc,0.64,0.61,0.67
5,vradobs,km s^{-1},22.58,6.42,50.38
6,alpha,degree,5.22,1.73,9.97
7,v3dobs,km s^{-1},273.78,76.82,852.56
8,d3d,Mpc,0.65,0.62,0.68
9,v3dcol,km s^{-1},1292.27,1199.54,1532.60
